In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import numpy as np
import statsmodels.api as sm
from io import StringIO

import preprocessor as pp
import emoji
from nltk.tokenize import TweetTokenizer



In [2]:
df = pd.read_csv('../data/monkeypox.csv')
df = df.drop('number', axis=1)
df = df.drop('created_at', axis=1)


In [3]:
# Format text columns
text_columns = ['text', 'user description'] 

for cl in text_columns:
    df[cl] = [emoji.demojize(pp.tokenize(text)).lower() for text in df[cl]]

In [4]:
# Check if the values needs to be standadized
for cl in df.columns:
    if cl not in text_columns:
        print(cl)
        print(df[cl].unique())


source
['Twitter Web App' 'TweetDeck' 'eClincher' 'Sprout Social'
 'SocialNewsDesk' 'Twitter for iPhone' 'WordPress.com'
 'Monkeypox News & Data' 'Hootsuite Inc.' 'Twitter for Android'
 'TwitterRSSTSOLAgent' 'NewsWorldpress' 'aa.com.tr' 'Tweetbot for iΟS'
 'SocialFlow' 'IFTTT' 'Brandwatch' 'dlvr.it' 'Twitter for iPad'
 'San Francisco PLOW' 'Zapier.com' 'Microsoft Power Platform'
 'Circleboom Publish' 'True Anthem' 'Insatiable Retriever'
 'HelloworldTest123' 'AI' 'Echobox' 'Twitter' 'Sprinklr Publishing'
 'The Tweeted Times' 'Twidere for Android' 'VernonNow' 'ShuswapNow'
 'PentictonNow' 'Kamloops BC Now' 'Welcome 2 Kelowna News'
 'PrinceGeorgeNow' 'VictoriaNow' 'FS_Poster_App'
 'Twitter Media Studio - LiveCut' 'Buffer' 'Revive Social App'
 'br00t4c App 01' 'skiviers' 'presshub_usbot' 'Next Newsfeeds.media'
 'NewsNacho' 'theglobe.co.com' 'Twitter Media Studio'
 'TweetCaster for iOS' 'Akhbaralyawm.com Post' 'POZ Auto Tweet'
 'RH Auto Tweet' 'TS Auto Tweet' 'ContentStudio.io' 'Germany News

In [5]:
for cl in df.columns:
    if cl not in text_columns:
        print(cl)
        print(df[cl].value_counts())


source
Twitter for iPhone     1780
Twitter Web App        1613
Twitter for Android    1095
WordPress.com           357
Twitter for iPad        181
                       ... 
Chronicle.lu v2           1
newslink.app              1
The Global Herald         1
Mirage News Posts         1
SMAP Lite                 1
Name: source, Length: 148, dtype: int64
user is verified
False    5385
True      402
Name: user is verified, dtype: int64
user has url
False    3566
True     2221
Name: user has url, dtype: int64
user created at
2013-08-13 06:16:35    44
2018-02-02 14:55:32    24
2020-03-11 08:05:33    22
2022-01-09 11:48:24    21
2022-01-14 23:40:55    21
                       ..
2022-04-17 19:41:24     1
2022-05-15 15:48:14     1
2016-08-20 18:11:59     1
2020-02-25 23:47:56     1
2019-09-18 11:53:12     1
Name: user created at, Length: 4588, dtype: int64
retweet_count
0      4575
1       550
2       176
3       104
4        73
       ... 
113       1
343       1
185       1
66        1
166

In [6]:
print(df['binary_class'].value_counts())

print(df['ternary_class'].value_counts())


0    4718
1    1069
Name: binary_class, dtype: int64
9    2753
0    1965
1    1069
Name: ternary_class, dtype: int64


In [7]:
tweet_tokenizer = TweetTokenizer()

for cl in text_columns:
    df[cl] = [tweet_tokenizer.tokenize(text) for text in df[cl]]



In [8]:
df

,text,source,user is verified,user has url,user description,user created at,retweet_count,reply_count,like_count,quote_count,followers count,following count,tweet count,listed_count,user location,binary_class,ternary_class
0,"[much, of, the, focus, on, $, hashtag, $, rece...",Twitter Web App,False,False,"[mother, of, $, number, $, ,, ex, wife, of, $,...",2011-08-14 19:22:06,2,0,2,0,3696,2993,3459,25,"London, England",0,0
1,"[icymi, :, the, first, probable, case, of, mon...",TweetDeck,True,True,"[wchs-tv, serves, the, charleston-huntington, ...",2009-04-17 15:21:43,2,0,1,0,75836,1753,125528,531,"Charleston, WV",0,0
2,"[who, :, $, hashtag, $, outbreak, not, yet, a,...",eClincher,False,False,"[patient, engagement, |, revenue, cycle, manag...",2019-03-05 19:24:33,0,0,0,0,2852,2381,10615,14,"Utah, USA",0,0
3,"[according, to, the, cdc, ,, monkeypox, is, us...",Sprout Social,True,True,"[home, of, the, minnesota, $, mention, $, and,...",2009-02-18 14:57:13,1,0,1,0,22897,5665,119258,663,Downtown Minneapolis,0,0
4,"[lgbtq, advocates, and, health, care, organiza...",SocialNewsDesk,True,True,"[spectrum, news, $, number, $, offers, the, be...",2008-10-27 02:14:13,0,0,0,0,81296,466,148219,747,"Rochester, NY",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5782,"[$, mention, $, $, mention, $, $, mention, $, ...",Twitter for Android,False,False,[none],2022-06-25 04:07:19,0,0,0,0,4,199,394,0,None,1,1
5783,"[$, mention, $, $, mention, $, $, mention, $, ...",Twitter for Android,False,False,[none],2022-06-25 04:07:19,0,0,0,0,4,199,394,0,None,1,1
5784,"[$, mention, $, it, seems, we, have, a, new, v...",Twitter Web App,False,False,"[hermit, who, loves, to, read, and, has, $, nu...",2020-03-21 13:40:39,0,0,2,0,66,201,7032,0,None,1,1
5785,"[$, mention, $, from, what, i, gather, ', monk...",Twitter for Android,False,False,"[a, well, educated, mind, will, always, have, ...",2022-03-04 11:46:18,0,0,0,0,354,2350,3066,1,None,1,1
